Допустим, у нас есть $n$ товаров с заданными стоимостями $v_i$ и массой $w_i$. В сумку убирается $С$ кг. Сколько какого товара взять, чтобы сумма всех стоимостей товаров была наибольшей?

In [1]:
values = [4, 2, 1, 7, 3, 6]
weights = [5, 9, 8, 2, 6, 5]
C = 15
n = 6

Сформулируем задачу:
$$\max\sum v_i x_i$$
$$\sum w_i x_i \leq C $$

Как должна выглядеть задача:
$$\min c^T x$$
$$A x \leq b $$

Получается, что $c=-v$, $A=w^T$, $b=(C)$

In [2]:
import numpy as np

In [3]:
c = - np.array(values)
A = np.array(weights)         #shape = (6,)
A = np.expand_dims(A, 0)      #shape = (1,6)
b = np.array([C])

In [4]:
from scipy.optimize import linprog

In [5]:
linprog(c=c, A_ub=A, b_ub=b)

     con: array([], dtype=float64)
     fun: -52.50000000003077
 message: 'Optimization terminated successfully.'
     nit: 5
   slack: array([-2.2495783e-11])
  status: 0
 success: True
       x: array([6.18738529e-14, 1.05853305e-12, 1.21475943e-13, 7.50000000e+00,
       4.00246690e-13, 4.71394160e-13])

In [6]:
!pip install cvxpy

In [7]:
!pip install cvxopt

In [6]:
import cvxpy

In [7]:
x = cvxpy.Variable(shape=n, integer = True)

In [8]:
constraint = (A @ x <= b)
total_value = c @ x

In [9]:
problem = cvxpy.Problem(cvxpy.Minimize(total_value), constraints=[constraint])

In [10]:
problem.solve(solver='ECOS_BB')

-138412039.00000018

In [11]:
x.value

array([ 8388608.00000001, -8388608.00000001, -8388608.00000001,
        8388608.00000001,  6990509.00000001,  8388608.00000001])

Теперь положительные $x$

In [12]:
x = cvxpy.Variable(shape=n, integer=True)
constraint = (A @ x <= b)
x_positive = (x >= 0)
total_value = c @ x
problem = cvxpy.Problem(cvxpy.Minimize(total_value), constraints=[constraint, x_positive])
problem.solve(solver='ECOS_BB')

/Users/Ivan/opt/anaconda3/lib/python3.8/site-packages/cvxpy/problems/problem.py:1267: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  warnings.warn(


-49.00000001590603

In [13]:
x.value

array([7.01265807e-10, 7.99333027e-10, 3.58703130e-10, 7.00000000e+00,
       4.67143021e-10, 9.34955115e-10])

Теперь $x = 0$ или $1$

In [14]:
x = cvxpy.Variable(shape=n, boolean=True)
constraint = A @ x <= b
x_positive = x >= 0
total_value = c @ x
problem = cvxpy.Problem(cvxpy.Minimize(total_value), constraints=[constraint, x_positive])
problem.solve(solver='ECOS_BB')

-17.00000000382157

In [15]:
x.value

array([1.00000000e+00, 2.25474413e-10, 2.07396078e-10, 1.00000000e+00,
       2.24003299e-10, 1.00000000e+00])

Составьте оптимальный план перевозок, со Склада № 1 и Склада № 2, в три торговых центра, с учётом тарифов, запасов и потребностей, которые указаны в таблице:

In [16]:
import numpy as np
import cvxpy
c = np.array([[2, 5, 3], [7, 7, 6]])
x = cvxpy.Variable(shape=(2, 3), integer=True)
constraint = [cvxpy.sum(x[0]) <= 180, 
              cvxpy.sum(x[1]) <= 220, 
              cvxpy.sum(x[:, 0]) == 110, 
              cvxpy.sum(x[:, 1]) == 150, 
              cvxpy.sum(x[:, 2]) == 140, 
              x >= 0]
total_value = cvxpy.sum(cvxpy.multiply(c, x))
problem = cvxpy.Problem(cvxpy.Minimize(total_value), constraints=constraint)
problem.solve(solver='ECOS_BB')

1900.0000000102357

Задача о назначениях

In [17]:
c = np.array([[1000, 12, 10, 19, 8],
              [12, 1000, 3, 7, 2],
              [10, 3, 1000, 6, 20],
              [19, 7, 6, 1000, 4],
              [8, 2, 20, 4, 1000]])
x = cvxpy.Variable(shape=(5,5), boolean=True)
constraint1 = [cvxpy.sum(x, axis=1) == np.ones(5),
               cvxpy.sum(x, axis=0) == np.ones(5)]
func = cvxpy.sum(cvxpy.multiply(x, c))
problem = cvxpy.Problem(cvxpy.Minimize(func), constraints=constraint1)
problem.solve(solver='ECOS_BB')

31.999999999961712

In [18]:
a = np.ones(5)

In [19]:
a

array([1., 1., 1., 1., 1.])

Коммивояжер, топология звезда, вернуться в первую точку.

In [25]:
c = np.array([[0, 12, 10, 19, 8],
              [12, 0, 3, 7, 2],
              [10, 3, 0, 6, 20],
              [19, 7, 6, 0, 4],
              [8, 2, 20, 4, 0]])
x = cvxpy.Variable(shape=(5,5), boolean=True)
u = cvxpy.Variable(shape=5, integer=True)
from itertools import product
constraints1 = [
    cvxpy.sum(x, axis=0) == np.ones(5),
    cvxpy.sum(x, axis=1) == np.ones(5),
    u >= 0,
    u <= 4,
    cvxpy.sum(cvxpy.diag(x)) == 0
]
for i, j in product(range(5), range(5)):
    if i >= 0 and j >= 1 and i != j:
        constraints1.append(u[i] - u[j] + 5 * x[i,j] <= 4)

func = cvxpy.sum(cvxpy.multiply(x, c))
problem = cvxpy.Problem(cvxpy.Minimize(func), constraints=constraints1)
problem.solve(solver='ECOS_BB')

31.99999994152333

In [4]:
a = [20, 23, 29, 22, 50, 43, 35] 
b = [70, 65, 58, 90, 45, 57, 50]
from scipy.stats import linregress 
linregress(a, b)

LinregressResult(slope=-1.0003628447024673, intercept=93.86865021770683, rvalue=-0.7694801436457975, pvalue=0.043089522157709934, stderr=0.3713232562153024)

In [7]:
import pandas as pd 
a = pd.Series([20, 23, 29, 22, 50, 43, 35, 98, 28]) 
b = pd.Series([70, 65, 58, 90, 45, 57, 50, 90, 38]) 
a.corr(b)

0.3138446778421647